In [3]:
import pandas as pd
import statistics
from sklearn.feature_extraction.text import CountVectorizer



In [4]:
vectorizer=CountVectorizer()

In [ ]:
vectorizer.

In [2]:
import math

In [3]:
ln=math.log

In [4]:
fake_users = pd.read_csv("../data/raw/BAZREVISEDPERSONAS.csv",delimiter=";")
fake_users.head()
fake_users.index+=1

In [5]:
big5=["Openness","Conscientiousness","Extraversion","Agreeableness","Neuroticism"]


for i in big5:
    fake_users[i] *= 0.01

In [6]:
fake_users["Neuroticism"] = 1-fake_users["Neuroticism"]

In [9]:
from itertools import islice


class Recommendation:
    def __init__(self,user_id,df):
        self.big5=["Extraversion","Agreeableness","Conscientiousness","Neuroticism","Openness"]
        self.importance_index={"Openness":0.7,"Extraversion":0.3,"Agreeableness":0.5,"Neuroticism":0.4,"Conscientiousness":0.6}
        self.reverse_threshold=50    
        self.df=df
        self.overall_scores={}
        self.user_id=user_id


    
    def get_recommendations_list(self,n):
        """Creates sorted recommendation list and returns to the final user.

        Args:
            n(int): A number of recommended matches.

        Returns:
            sorted dictionary with similarity scores between target user and each other user

        """

        
        recommendation = self.main_recommender()
        sorted_recommendation = self.sorted_dictionary(recommendation)
        return self.take(n,sorted_recommendation.items())

    def take(self,n, iterable):

        
        """Return first n items of the iterable as a list.

        Args:
            n(int): The number of items in the dictionary needed.
            iterable(dictionary): A sorted overall ratings dictionary.

        Returns:
            A sliced dictionary.

        """
  
        return list(islice(iterable, n))
        


    def sorted_dictionary(self,dictionary):       
        """Sorts dictionary by ratings ([0,1]).

        Args:

            dictionary(dictionary): An unsorted overall ratings dictionary.

        Returns:
            A sorted by rating dictionary .

        """
        
        return {k: v for k, v in sorted(dictionary.items(), key = lambda item: item[1],reverse = True)}
    def main_recommender(self):
        """ Calling personality, language and skills analysis methods to create a match rating score

        Returns:
            Overall ratings scores dictionary.

        """
        # load the dataset 
        self.personality_language_age()
        self.skills_analyser()
        return self.overall_scores
    
    def personality_language_age(self):
        """ Adds personality and language matching score
        Returns: 
            A modified overall scores with add personality and language {user_index:mean_difference}
        """
        # Split the language
        example_user_language = fake_users.loc[self.user_id]["Language"]



        user_qualities = {}
        # Filling our user qualities
        for i in self.big5:
            user_qualities[i] = self.df.loc[self.user_id][i]
        users_mean_difference = []

        for index, row in self.df.iterrows():
            mean_difference = 0
            if index != self.user_id:
                
                
                # Create the list of the user languages
                current_user_language = row["Language"]

                for i in self.big5:
                    row_trait = row[i]
                    users_trait = user_qualities[i]
                    difference = 0
                    #Keep track of counted rows
                    counted = False

                    # Check if the user quality is more than 40. Else, make a reverce relationship
#                     if user_qualities[i]<40 or row[i]<40:
                        
#                         difference+=abs(ln(row_trait/users_trait))*self.importance_index[i]
#                         mean_difference+=(1-difference)
#                         counted=True
                    if not counted:

                        mean_difference += abs(ln(row_trait/users_trait))*self.importance_index[i]

                mean_difference /= len(self.big5)
                
                #Store the final reverse mean difference (the bigger the better) in the dictionary
                self.overall_scores[index] = (1-mean_difference)

                
        return self.overall_scores
    
    def skills_analyser(self):
        """ Adds skills reciprocity to an overall matching score
        Returns: 
            A modified with skills reciprocity scores overall scores 
        """
        example_user_skills = self.df.loc[self.user_id]["Skills"].split(",")
        looking_for_skills = self.df.loc[self.user_id]["Looking for"].split(",")


        # get the targeted skills lost
        targeted_skills = []
        # Delete the final comma 
        del example_user_skills[-1]
        user_skills_list = []      
        for index,row in fake_users.iterrows():
            
            if index != self.user_id:
                current_user_skills = row["Skills"].split(",")
                current_user_looking_for = row["Looking for"].split(",")

                
                del current_user_skills[-1] # Delete the final comma
                
                #Finding the similarities between two lists
                similarities_xy = set(looking_for_skills) & set(current_user_skills)
                similarities_yx = set(current_user_looking_for) & set(example_user_skills) 
                #Calculating a similarity percentage(score)
                similarity_xy = len(similarities_xy)/len(looking_for_skills)
                similarity_yx = len(similarities_yx)/len(current_user_looking_for)
                similarities = statistics.mean([similarity_xy,similarity_yx])
                user_skills_list.append([index,similarities])
                self.overall_scores[index] += similarities
                


        return self.overall_scores


example_user_number = int(input("Please enter the user_number: "))
recommender = Recommendation(example_user_number,fake_users)
print("The best matches for user {} are:\n".format(recommender.user_id)+ str(recommender.get_recommendations_list(9)))

Please enter the user_number: 8
The best matches for user 8 are:
[(3, 0.8881298369060315), (10, 0.8645311425790803), (1, 0.8440084612896078), (4, 0.7787987964520333), (7, 0.7212515175246317), (2, 0.6701766380034315), (6, 0.6496653350663693), (5, 0.617483322707487), (9, 0.6082350942223472)]
